In [ ]:
import sys
import os
from pathlib import Path

# Add the root directory to the Python path
root_dir = os.path.abspath(Path(os.path.dirname(os.path.abspath("__file__"))).parent)
sys.path.append(root_dir)

print(f"Added {root_dir} to the Python path.")

In [ ]:
import os
import numpy as np
import pydicom
import plotly.graph_objects as go
from pydicom.pixel_data_handlers.util import apply_voi_lut
from src.config import *

# Function to load DICOM series and convert to 3D array
def load_dicom_series(directory):
    dicom_files = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(".dcm"):
                dicom_files.append(pydicom.dcmread(os.path.join(root, file)))
    dicom_files.sort(key=lambda x: int(x.InstanceNumber))
    
    # Stack all slices into a 3D numpy array
    image_3d = np.stack([apply_voi_lut(d.pixel_array, d) for d in dicom_files])
    
    # Normalize the data for visualization
    image_3d = image_3d.astype(np.float32)
    image_3d = (image_3d - np.min(image_3d)) / (np.max(image_3d) - np.min(image_3d)) * 255  # Normalize to [0, 255]
    
    return image_3d

# Example: Loading the series from your DICOM directory
volume = load_dicom_series(TRAIN_IMAGES_PATH / EXAMPLE_ID / EXAMPLE_SAGITTAL_T1_ID)

nb_frames, r, c = volume.shape

# Define frames for the animation
fig = go.Figure(frames=[go.Frame(data=go.Surface(
    z=(4.9 - k * (5 / nb_frames)) * np.ones((r, c)),
    surfacecolor=np.flipud(volume[nb_frames - 1 - k]),
    cmin=0, cmax=255  # Adjust cmin and cmax to match your normalized volume
    ),
    name=str(k)  # Name the frame for the animation to behave properly
    )
    for k in range(nb_frames)])

# Add initial data to be displayed before animation starts
fig.add_trace(go.Surface(
    z=4.9 * np.ones((r, c)),
    surfacecolor=np.flipud(volume[nb_frames - 1]),
    colorscale='Gray',
    cmin=0, cmax=255,  # Adjust to match the intensity range of the volume
    colorbar=dict(thickness=20, ticklen=4)
    ))

# Function for frame arguments
def frame_args(duration):
    return {
            "frame": {"duration": duration},
            "mode": "immediate",
            "fromcurrent": True,
            "transition": {"duration": duration, "easing": "linear"},
        }

# Slider configuration
sliders = [
            {
                #"pad": {"b": 10, "t": 60},
                "len": 0.9,
                "x": 0.1,
                "y": 0,
                "steps": [
                    {
                        "args": [[f.name], frame_args(0)],
                        "label": str(k),
                        "method": "animate",
                    }
                    for k, f in enumerate(fig.frames)
                ],
            }
        ]

# Layout configuration
fig.update_layout(
         title='Slices in volumetric DICOM data',
         width=600,
         height=600,
         scene=dict(
                    zaxis=dict(range=[-.2, 5.1], autorange=False),
                    aspectratio=dict(x=1, y=1, z=1),
                    ),
         updatemenus = [
            {
                "buttons": [
                    {
                        "args": [None, frame_args(15)],
                        "label": "&#9654;",  # play symbol
                        "method": "animate",
                    },
                    {
                        "args": [[None], frame_args(0)],
                        "label": "&#9724;",  # pause symbol
                        "method": "animate",
                    },
                ],
                "direction": "left",
                #"pad": {"r": 10, "t": 70},
                "type": "buttons",
                "x": 0.1,
                "y": 0,
            }
         ],
         sliders=sliders
)

# Display the interactive 3D animation
fig.show()

In [ ]:
# Import data
import time
import numpy as np

from skimage import io

vol = io.imread("https://s3.amazonaws.com/assets.datacamp.com/blog_assets/attention-mri.tif")
volume = vol.T
r, c = volume[0].shape

# Define frames
import plotly.graph_objects as go
nb_frames = 68

fig = go.Figure(frames=[go.Frame(data=go.Surface(
    z=(6.7 - k * 0.1) * np.ones((r, c)),
    surfacecolor=np.flipud(volume[67 - k]),
    cmin=0, cmax=200
    ),
    name=str(k) # you need to name the frame for the animation to behave properly
    )
    for k in range(nb_frames)])

# Add data to be displayed before animation starts
fig.add_trace(go.Surface(
    z=6.7 * np.ones((r, c)),
    surfacecolor=np.flipud(volume[67]),
    colorscale='Gray',
    cmin=0, cmax=200,
    colorbar=dict(thickness=20, ticklen=4)
    ))


def frame_args(duration):
    return {
            "frame": {"duration": duration},
            "mode": "immediate",
            "fromcurrent": True,
            "transition": {"duration": duration, "easing": "linear"},
        }

sliders = [
            {
                "pad": {"b": 10, "t": 60},
                "len": 0.9,
                "x": 0.1,
                "y": 0,
                "steps": [
                    {
                        "args": [[f.name], frame_args(0)],
                        "label": str(k),
                        "method": "animate",
                    }
                    for k, f in enumerate(fig.frames)
                ],
            }
        ]

# Layout
fig.update_layout(
         title='Slices in volumetric data',
         width=600,
         height=600,
         scene=dict(
                    zaxis=dict(range=[-0.1, 6.8], autorange=False),
                    aspectratio=dict(x=1, y=1, z=1),
                    ),
         updatemenus = [
            {
                "buttons": [
                    {
                        "args": [None, frame_args(50)],
                        "label": "&#9654;", # play symbol
                        "method": "animate",
                    },
                    {
                        "args": [[None], frame_args(0)],
                        "label": "&#9724;", # pause symbol
                        "method": "animate",
                    },
                ],
                "direction": "left",
                "pad": {"r": 10, "t": 70},
                "type": "buttons",
                "x": 0.1,
                "y": 0,
            }
         ],
         sliders=sliders
)

fig.show()

In [ ]:
print(vol.shape, volume.shape)